# ESP32 MicroPython Installation

This guide shows how to install MicroPython on an ESP32 microcontroller ([Adafruit Huzzah32](https://www.adafruit.com/product/3405) used here, but should work with other boards also) and programming it with it with the help of [Jupyter Notebooks](https://jupyterlab.readthedocs.io/en/stable/). 

What's a Jupyter Notebook? You are looking at one! A mix of cells containing code, explanations, images, etc. With the correct setup the Notebook is "alive", i.e. you can execute and modify the code in the notebook.

## Download the ESP32 Repository

First you need to download these instructions and related files. To do this, you need to install `git` on your computer. Follow the instructions on [https://git-scm.com](https://git-scm.com/download/).

Now open a terminal window on your computer (e.g. [cmd.exe](https://en.wikipedia.org/wiki/Cmd.exe) on Windows or [terminal.app](https://en.wikipedia.org/wiki/Terminal_(macOS)) on macOS) in the folder where you want to save your esp32 programs and run the following command:

```shell
git clone https://github.com/bboser/esp32.git
```


## Install Python 3

Since the ESP32 does not have a keyboard and display, we use the host computer to interact with it. For this we need an installation of Python version 3.6 or later on the host. 

The most recent version of Python can be downloaded from [python.org](https://www.python.org/downloads/). At least version 3.6 is required. If you are not sure which version you have, open a terminal window  and start the interpreter. 

On some systems, Python 3 is started with `python`, others use `python3`. Just try to see what works or check the documentation of your installation of Python:

```shell
$ python3
Python 3.6.5 (v3.6.5:f59c0932b4, Mar 28 2018, 05:52:31)
[GCC 4.2.1 Compatible Apple LLVM 6.0 (clang-600.0.57)] on darwin
Type "help", "copyright", "credits" or "license" for more information.
>>>
```

Also install a few libraries using the `pip` command (again, `pip3` or `pip`, depending on the setup):

```shell
pip3 install --upgrade pip
pip3 install pyserial
pip3 install esptool
pip3 install matplotlib
```

## Install Jupyter

Now we are ready to install `Jupyter` so this notebook becomes alive.

```shell
pip3 install jupyter
pip3 install jupyterlab
pip3 install jupyter-micropython-remote
```

Download this file and start Jupyter in the folder with the file:

```shell
jupyter lab
```

A browser window opens with these instructions (navigate the folders in the left pane if this file does not open automatically). Now the notebook is alive and can excecute Python code. To run the contents of a cell, hold down the shift key and press enter.

In [1]:
%local

for i in range(3):
    print(i**2)

0
1
4


The line with `%local` instructs Jupyter to run the code on the host computer rather than the ESP32. For that to work we need to install Python on the ESP32.

## Install MicroPython

The next step is to install Python on the ESP32. Since the ESP32 has little memory, a special version called `MicroPython` is used. It has the same syntax as regular Python, but lacks some libraries, e.g.  to interact with displays (it does not have one).

### Find the Serial Port Name

Connect the ESP32 board to a free USB port on your computer and find its port. On Linux and OSX this will be a "file" in `/dev`, e.g. `/dev/cu.SLAB_USBtoUART`. On Windows it's a `COM` port, e.g. `COM3`. You can find it by unplugging and replugging the board and checking which entry gets added to `/dev` or the Windows `COM` ports.

On some systems you can run the code below to get a list of candidates:

In [2]:
%local

from serial.tools.list_ports import comports

for port in comports():
    if port.vid and port.vid == 4292:
        print(f"Found board {port}")

Found board /dev/cu.SLAB_USBtoUART - CP2104 USB to UART Bridge Controller


### Flash MicroPython to the ESP32

The next step is to copy the MicroPython binary to the flash memory on the ESP32 board. Edit the serial port name and name of the binary below to match your setup. Then excecute the cell. 

In [3]:
%shell esptool.py -p /dev/cu.SLAB_USBtoUART --chip esp32 -b 921600 erase_flash
%shell esptool.py -p /dev/cu.SLAB_USBtoUART --chip esp32 -b 921600 write_flash -z 0x1000 bin/esp32-latest.bin

esptool.py v2.5.1
Serial port /dev/cu.SLAB_USBtoUART
Connecting........_
Chip is ESP32D0WDQ6 (revision 1)
Features: WiFi, BT, Dual Core
MAC: 30:ae:a4:1a:27:28
Uploading stub...
Running stub...
Stub running...
Changing baud rate to 921600
Changed.
Erasing flash (this may take a while)...
Chip erase completed successfully in 4.4s
Hard resetting via RTS pin...
esptool.py v2.5.1
Serial port /dev/cu.SLAB_USBtoUART
Connecting........__
Chip is ESP32D0WDQ6 (revision 1)
Features: WiFi, BT, Dual Core
MAC: 30:ae:a4:1a:27:28
Uploading stub...
Running stub...
Stub running...
Changing baud rate to 921600
Changed.
Configuring flash size...
Auto-detected Flash size: 4MB
Compressed 1081952 bytes to 683886...
Wrote 1081952 bytes (683886 compressed) at 0x00001000 in 9.5 seconds (effective 906.4 kbit/s)...
Hash of data verified.

Leaving...
Hard resetting via RTS pin...


The `esptool` prints some feedback about what it is doing and after a minute or two announces success with:

```shell
Leaving...
Hard resetting via RTS pin...
```

If you get errors follow the instructions [micropython.org](http://micropython.org/download#esp32) to flash MicroPython to your board.Now you are ready to run Python on the ESP32. To connect, execute the cell below (edit the port to match your setup). You should get a message `Connected on /dev/cu.SLAB_USBtoUART`.

In [3]:
%connect "/dev/cu.SLAB_USBtoUART"

Connected on /dev/cu.SLAB_USBtoUART


Check it out, e.g.

In [3]:
print("2 to the power 50:", 2**50)

import sys
print("platform:", sys.platform)
print("implementation:", sys.implementation)

2 to the power 50: 1125899906842624
platform: esp32
implementation: (name='micropython', version=(1, 9, 4))


The output `platform: esp32` confirms that we are indeed running Python on the microcontroller. Now you can sell your laptop ... hold off, you still need the display and keyboard.

# Blinking LEDs

To gain a bit more confidence that our board is indeed working, we will write a small program that flashes an LED. This program is also a good place to go back to when suddently everything stops working.

Run the cell below and experience the thrill of a blinking light! There is more to come ...

In [3]:
from machine import Pin
from time import sleep

led = Pin(13, mode=Pin.OUT)

for i in range(3):
    led.value(1)
    print("LED on ", i)
    sleep(0.5)
    led.value(0)
    print("LED off", i)
    sleep(0.2)

LED on  0
LED off 0
LED on  1
LED off 1
LED on  2
LED off 2


# Documentation

Now we have a pretty powerful setup, but how do we use it? We will build up our knowledge over time, but here are a few links to reference documentation to go back to. 

Often a generic web search is the best path to getting help. Also typing error messages into a web search engine frequently finds a solution.

**A Note about the Open Source Community**

*MicroPython and other software we use is written by volunteers and made available to everyone to use for free. Frequently the documentation is not up-to-date. If you find something needs improvement, the community is always looking for contributions.*

## Jupyter

* General documentation about [Jupyter Notebooks](https://jupyterlab.readthedocs.io/en/stable/)
* Documentation for the [MicroPython Kernel](https://pypi.org/project/jupyter-micropython-remote/) for Jupyter

## MicroPython

* General documentation on [MicroPython](http://docs.micropython.org/en/latest/)
* Documentation for the ESP32 is not available, but the instructions for the [ESP8266](http://docs.micropython.org/en/latest/esp8266/quickref.html) mostly apply. The ESP8266 is the predecessor of the ESP32.